## GSAT trend patterns

In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
# import src.slurm_cluster as scluster
# client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.mk_test(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val
 

### loaded the separated data from models

In [ ]:
dir1 ='/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure2/ACCESS/'
ACCESS_forced = xr.open_mfdataset(dir1 + 'GSAT_ACCESS_Forced_anomalies_1850_2022.nc',chunks={'run': 1})
ACCESS_internal = xr.open_mfdataset(dir1 + 'GSAT_ACCESS_Internal_Variability_anomalies_1850_2022.nc', chunks={'run': 1})

In [ ]:
ACCESS_internal

In [ ]:
ACCESS_forced = ACCESS_forced.rename({'__xarray_dataarray_variable__':'tas'})

In [ ]:
ACCESS_forced

### Calculate the trend on 10-73yrs length with end year anchoring at 2022

In [ ]:
import pandas as pd
import numpy as np

start_year = 1950
end_year = 2022
min_length = 10

In [ ]:
years = np.arange(start_year, end_year + 1)

# Prepare a dictionary to hold the trend DataArrays for each period
trend_data = {}
p_value_data = {}

for begin_year in range(start_year, end_year - min_length+2):
    # Select the data from the start year to the last year
    time_slice = ACCESS_forced.sel(year=slice(begin_year, end_year))
    
    # Calculate the trend for each grid cell
    trend, p_values = xr.apply_ufunc(
        func_mk,   # Pass the appropriate years
        time_slice,
        input_core_dims=[["year"]],
        output_core_dims=[[], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float, float],
        dask_gufunc_kwargs={'allow_rechunk': True}
    )
    period_key = f'{begin_year}-{end_year}'
    trend_data[period_key] = trend
    p_value_data[period_key] = p_values

In [ ]:
trend_data

In [ ]:
# Convert dictionaries to xarray DataArray for saving
trend_da = xr.concat(trend_data.values(), dim=pd.Index(trend_data, name='period'))


In [ ]:
p_value_da = xr.concat(p_value_data.values(), dim=pd.Index(p_value_data, name='period'))

In [ ]:
trend_da['period']

In [ ]:
trend_da.sel(period='1950-2022')

In [ ]:
# trend_annual_np = {}
# pvalue_annual_np = {}

# for i in trend_data.keys():
#     trend_annual_np[i] = trend_data[i].values
#     pvalue_annual_np[i] = p_value_data[i].values

In [ ]:
# trend_annual_da = {}
# pvalue_annual_da = {}

# for interval, data in trend_annual_np.items():
#     trend_annual_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": ACCESS_forced_data_dict[interval].run, 
#     "lat": ACCESS_forced_data_dict[interval].lat, "lon": ACCESS_forced_data_dict[interval].lon})
# for interval, data in pvalue_annual_np.items():
#     pvalue_annual_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": ACCESS_forced_data_dict[interval].run, 
#     "lat": ACCESS_forced_data_dict[interval].lat, "lon": ACCESS_forced_data_dict[interval].lon})

In [ ]:
# out put the forced trend and pvalue
dir_out ='/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/LE_evaluation/Fig3_ACCESS/output/'

trend_da.to_netcdf(dir_out + 'ACCESS_forced_segmented_trend.nc')
# p_value_xr.to_netcdf(dir_out + 'ACCESS_forced_segmented_pvalue.nc')

In [ ]:
trend_da

### Plotting with the Robinson Projections

In [ ]:
plt.rcParams['figure.figsize'] = (8, 10)
plt.rcParams['font.size'] = 16
# plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['ytick.major.right'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.bottom'] = True
plt.rcParams['savefig.transparent'] = True # save the figure with a transparent background
x = np.arange(1850, 2015, 1)

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm, ListedColormap

In [ ]:
def plot_trend(trend_data, lats, lons, levels=None, extend=None, cmap=None, title="", ax=None, show_xticks=False, show_yticks=False):
    """
    Plot the trend spatial pattern using Robinson projection with significance overlaid.

    Parameters:
    - trend_data: 2D numpy array with the trend values.
    - lats, lons: 1D arrays of latitudes and longitudes.
    - p_values: 2D array with p-values for each grid point.
    - GMST_p_values: 2D array with GMST p-values for each grid point.
    - title: Title for the plot.
    - ax: Existing axis to plot on. If None, a new axis will be created.
    - show_xticks, show_yticks: Boolean flags to show x and y axis ticks.
    
    Returns:
    - contour_obj: The contour object from the plot.
    """
    # Create a new figure/axis if none is provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(20, 15), subplot_kw={'projection': ccrs.Robinson()})
        ax.set_global()

    # Determine significance mask (where p-values are less than 0.05)
    # insignificance_mask = p_values >= 0.10
    # Plotting
    # contour_obj = ax.pcolormesh(lons, lats, trend_data,  cmap='RdBu_r',vmin=-5.0, vmax=5.0, transform=ccrs.PlateCarree(central_longitude=180), shading='auto')
    contour_obj = ax.contourf(lons, lats, trend_data, levels=levels, extend=extend, cmap=cmap, transform=ccrs.PlateCarree(central_longitude=0))

    # Plot significance masks with different hatches
    # ax.contourf(lons, lats, insignificance_mask, levels=[0.0, 0.10, 1.5],hatches=[None,'///'], colors='none', transform=ccrs.PlateCarree())

    ax.coastlines(resolution='110m')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,
                      colors='gray', alpha=0.5, linestyle='--', linewidth=0.5)

    # Disable labels on the top and right of the plot
    gl.top_labels = False
    gl.right_labels = False

    # Enable labels on the bottom and left of the plot
    gl.bottom_labels = show_xticks
    gl.left_labels = show_yticks
    gl.xformatter = cticker.LongitudeFormatter()
    gl.yformatter = cticker.LatitudeFormatter()
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}
    
    if show_xticks:
        gl.bottom_labels = True
    if show_yticks:
        gl.left_labels = True
    
    ax.set_title(title, loc='center', fontsize=18, pad=5.0)

    return contour_obj

In [ ]:
def plot_trend_with_significance(trend_data, lats, lons, p_values, GMST_p_values=None, levels=None, extend=None, cmap=None, title="", ax=None, show_xticks=False, show_yticks=False):
    """
    Plot the trend spatial pattern using Robinson projection with significance overlaid.

    Parameters:
    - trend_data: 2D numpy array with the trend values.
    - lats, lons: 1D arrays of latitudes and longitudes.
    - p_values: 2D array with p-values for each grid point.
    - GMST_p_values: 2D array with GMST p-values for each grid point.
    - title: Title for the plot.
    - ax: Existing axis to plot on. If None, a new axis will be created.
    - show_xticks, show_yticks: Boolean flags to show x and y axis ticks.
    
    Returns:
    - contour_obj: The contour object from the plot.
    """
    # Create a new figure/axis if none is provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(20, 15), subplot_kw={'projection': ccrs.Robinson()})
        ax.set_global()

    # Determine significance mask (where p-values are less than 0.05)
    insignificance_mask = p_values >= 0.10
    # Plotting
    # contour_obj = ax.pcolormesh(lons, lats, trend_data,  cmap='RdBu_r',vmin=-5.0, vmax=5.0, transform=ccrs.PlateCarree(central_longitude=180), shading='auto')
    contour_obj = ax.contourf(lons, lats, trend_data, levels=levels, extend=extend, cmap=cmap, transform=ccrs.PlateCarree(central_longitude=0))

    # Plot significance masks with different hatches
    ax.contourf(lons, lats, insignificance_mask, levels=[0.0, 0.10, 1.5],hatches=[None,'///'], colors='none', transform=ccrs.PlateCarree())

    ax.coastlines(resolution='110m')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,
                      colors='gray', alpha=0.5, linestyle='--', linewidth=0.5)

    # Disable labels on the top and right of the plot
    gl.top_labels = False
    gl.right_labels = False

    # Enable labels on the bottom and left of the plot
    gl.bottom_labels = show_xticks
    gl.left_labels = show_yticks
    gl.xformatter = cticker.LongitudeFormatter()
    gl.yformatter = cticker.LatitudeFormatter()
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}
    
    if show_xticks:
        gl.bottom_labels = True
    if show_yticks:
        gl.left_labels = True
    
    ax.set_title(title, loc='center', fontsize=18, pad=5.0)

    return contour_obj

In [ ]:
dir_input = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/LE_evaluation/Fig3_ACCESS/output/'
trend_da = xr.open_mfdataset(dir_input + 'ACCESS_forced_segmented_trend.nc', chunks={'run': 1})

In [ ]:
trend_da

In [ ]:
lat = trend_da['lat']
lon = trend_da['lon']

In [ ]:
lat

In [ ]:
# define an asymmetric colormap
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.colors import BoundaryNorm
import cartopy.util as cutil
import seaborn as sns
import matplotlib.colors as mcolors
import palettable

In [ ]:
# title is from realization 1 to realization 21
titles = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 
          'r11', 'r12', 'r13', 'r14', 'r15', 'r16', 'r17', 'r18', 'r19',
          'r20', 'r21', 'r22', 'r23', 'r24', 'r25', 'r26', 'r27', 'r28',
          'r29', 'r30', 'r31', 'r32', 'r33', 'r34', 'r35', 'r36', 'r37',
          'r38', 'r39', 'r40']
trend_annual_data = trend_da.sel(period='2013-2022')*10.0

levels = np.arange(-0.6, 0.65, 0.05)
extend = 'both'

In [ ]:
type(trend_annual_data)

In [ ]:
trend_annual_data['tas']

In [ ]:
p_value_data = p_value_da.sel(period='2013-2022')

In [ ]:
p_value_data

In [ ]:
print(trend_annual_data.dims)
# print(pvalue_annual_data.dims)

In [ ]:
# check the min and max value of the trend
print(trend_annual_data.min().values)
print(trend_annual_data.max().values)

### Plotting

In [ ]:
# Plot the trend patterns 
def plot_data(ax, title):
    run_number = int(title.lstrip('r')) # Strip 'r' and convert to integer if necessary
    print(run_number)
    
    trend_data = trend_annual_data['tas'].sel({'run': run_number}) 
    # p_values = p_value_data['tas'].sel({'run': run_number})
    
    trend_data_with_cyclic, lon_cyclic = cutil.add_cyclic_point(trend_data, coord=lon)
    # p_values_with_cyclic, _ = cutil.add_cyclic_point(p_values, coord=lon)
    
    ax.set_global()
    
    # Assuming plot_trend_with_significance is defined correctly and returns a contour object
    contour_obj = plot_trend(trend_data_with_cyclic, lat, lon_cyclic,
            levels=levels, extend=extend,
            cmap='twilight_shifted', title=" ",
            ax=ax, 
            show_xticks=False, 
            show_yticks=False)
    ax.text(0.5, 1.15, title, ha='center', va='center', transform=ax.transAxes)  # Adjusted to use transform
    ax.set_xticks([])
    ax.set_yticks([])
    return contour_obj

# Prepare for the loop
fig = plt.figure(figsize=(12, 15))
gs = gridspec.GridSpec(10, 5, wspace=0.05, hspace=0.05)

# Loop through and create subplots
for i, title in enumerate(titles):
    ax = fig.add_subplot(gs[i // 5, i % 5], projection=ccrs.Robinson(180))
    contour_obj=plot_data(ax, title)
    if i == 0:  # Arbitrarily choose the first plot's contour_obj for the colorbar
        contour_obj_for_cbar = contour_obj
        
# Assuming contour_obj is defined and consistent across plots
# Add horizontal colorbar
cbar_ax = fig.add_axes([0.25, 0.06, 0.6, 0.012])  # Adjust these values as needed
cbar = plt.colorbar(contour_obj, cax=cbar_ax, orientation='horizontal', extend='both')
cbar.ax.tick_params(labelsize=14)
cbar.set_label('Annual SAT Trend (°C/decade)', fontsize=16)

plt.tight_layout()
fig.savefig('ACCESS-[2013-2022]-10year-forced-trend.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
client.close()
scluster.close()

### Input the ensemble mean trend patterns; then calculate the pattern correlation

In [ ]:
# Input the 10-year, 30-year and 60-year ensemble mean SAT trend data
# dir_input = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Supp_Figure1_Forced/data/Smiles_ensemble/'

# ACCESS_10yr_trend   = xr.open_dataset(dir_input + 'ACCESS_annual_forced_10yr_trend.nc')
# ACCESS_30yr_trend   = xr.open_dataset(dir_input + 'ACCESS_annual_forced_30yr_trend.nc')
# ACCESS_60yr_trend   = xr.open_dataset(dir_input + 'ACCESS_annual_forced_60yr_trend.nc')

In [ ]:
dir_input = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Supp_Figure1_Forced/data/ACCESS_Single/'

ACCESS_10yr_trend   = xr.open_dataset(dir_input + 'ACCESS_annual_10yr_ensemble_mean_trend.nc')
ACCESS_30yr_trend   = xr.open_dataset(dir_input + 'ACCESS_annual_30yr_ensemble_mean_trend.nc')
ACCESS_60yr_trend   = xr.open_dataset(dir_input + 'ACCESS_annual_60yr_ensemble_mean_trend.nc')

In [ ]:
ACCESS_10yr_trend = ACCESS_10yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
ACCESS_30yr_trend = ACCESS_30yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
ACCESS_60yr_trend = ACCESS_60yr_trend.rename({'__xarray_dataarray_variable__':'tas'})

In [ ]:
# pattern correlation betwenn observed forced pattern vs. Model simulated forced pattern
import scipy.stats as stats
# 10yr forced trend in each realization .vs. ACCESS ensemble trend ACCESS_10yr_trend
ACCESS_forced_10yr_trend = trend_annual_da['10yr']
ACCESS_forced_30yr_trend = trend_annual_da['30yr']
ACCESS_forced_60yr_trend = trend_annual_da['60yr']

ACCESS_unforced_10yr_trend = unforced_trend_da['10yr']
ACCESS_unforced_30yr_trend = unforced_trend_da['30yr']
ACCESS_unforced_60yr_trend = unforced_trend_da['60yr']

# Pattern correlations calculation
trend_pattern_correlation_10yr = []
for i in range(len(trend_annual_da['10yr'].run)):
    trend_pattern_correlation_10yr.append(stats.pearsonr(ACCESS_forced_10yr_trend.isel(run=i).values.flatten(), ACCESS_10yr_trend['tas'].values.flatten())[0])

trend_pattern_correlation_30yr = []
for i in range(len(trend_annual_da['30yr'].run)):
    trend_pattern_correlation_30yr.append(stats.pearsonr(ACCESS_forced_30yr_trend.isel(run=i).values.flatten(), ACCESS_30yr_trend['tas'].values.flatten())[0])

trend_pattern_correlation_60yr = []
for i in range(len(trend_annual_da['60yr'].run)):
    trend_pattern_correlation_60yr.append(stats.pearsonr(ACCESS_forced_60yr_trend.isel(run=i).values.flatten(), ACCESS_60yr_trend['tas'].values.flatten())[0])
    

In [ ]:
trend_pattern_correlation_10yr

In [ ]:
with open('pattern_correlations_vs_mean_of_trend.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [ ]:
# save the pattern correlation into text file
# Save to a text file
with open('pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [ ]:
# Input the observational forced and unforced trend wrt. ACCESS-CM6A-LR
dir_obs_ACCESS = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure2/data/'

Obs_ACCESS_forced_10yr_trend = xr.open_dataset(dir_obs_ACCESS + 'HadCRUT5_annual_forced_10yr_trend_ACCESS.nc')
Obs_ACCESS_forced_30yr_trend = xr.open_dataset(dir_obs_ACCESS + 'HadCRUT5_annual_forced_30yr_trend_ACCESS.nc')
Obs_ACCESS_forced_60yr_trend = xr.open_dataset(dir_obs_ACCESS + 'HadCRUT5_annual_forced_60yr_trend_ACCESS.nc')

Obs_ACCESS_unforced_10yr_trend = xr.open_dataset(dir_obs_ACCESS + 'HadCRUT5_annual_internal_10yr_trend_ACCESS.nc')
Obs_ACCESS_unforced_30yr_trend = xr.open_dataset(dir_obs_ACCESS + 'HadCRUT5_annual_internal_30yr_trend_ACCESS.nc')
Obs_ACCESS_unforced_60yr_trend = xr.open_dataset(dir_obs_ACCESS + 'HadCRUT5_annual_internal_60yr_trend_ACCESS.nc')

In [ ]:
Obs_ACCESS_unforced_10yr_trend

In [ ]:
Obs_ACCESS_forced_10yr_trend = Obs_ACCESS_forced_10yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_ACCESS_forced_30yr_trend = Obs_ACCESS_forced_30yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_ACCESS_forced_60yr_trend = Obs_ACCESS_forced_60yr_trend.rename({'__xarray_dataarray_variable__':'tas'})

Obs_ACCESS_unforced_10yr_trend = Obs_ACCESS_unforced_10yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_ACCESS_unforced_30yr_trend = Obs_ACCESS_unforced_30yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_ACCESS_unforced_60yr_trend = Obs_ACCESS_unforced_60yr_trend.rename({'__xarray_dataarray_variable__':'tas'})

In [ ]:
# calculate the pattern correlation between observed and model's each realization simulated forced and unforced trend
OBS_trend_pattern_correlation_10yr = []
for i in range(len(trend_annual_da['10yr'].run)):
    print(i)
    OBS_trend_pattern_correlation_10yr.append(stats.pearsonr(ACCESS_forced_10yr_trend.isel(run=i).values.flatten(), Obs_ACCESS_forced_10yr_trend['tas'].values.flatten())[0])

OBS_trend_pattern_correlation_30yr = []
for i in range(len(trend_annual_da['30yr'].run)):
    print(i)
    OBS_trend_pattern_correlation_30yr.append(stats.pearsonr(ACCESS_forced_30yr_trend.isel(run=i).values.flatten(), Obs_ACCESS_forced_30yr_trend['tas'].values.flatten())[0])

OBS_trend_pattern_correlation_60yr = []
for i in range(len(trend_annual_da['60yr'].run)):
    print(i)
    OBS_trend_pattern_correlation_60yr.append(stats.pearsonr(ACCESS_forced_60yr_trend.isel(run=i).values.flatten(), Obs_ACCESS_forced_60yr_trend['tas'].values.flatten())[0])

In [ ]:
# Save to a text file
with open('obsForced_ACCESSforced_pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [ ]:
OBS_trend_pattern_correlation_10yr

In [ ]:
OBS_unforced_trend_pattern_correlation_10yr = []
for i in range(len(trend_annual_da['10yr'].run)):
    OBS_unforced_trend_pattern_correlation_10yr.append(stats.pearsonr(ACCESS_unforced_10yr_trend.isel(run=i).values.flatten(), Obs_ACCESS_unforced_10yr_trend['tas'].values.flatten())[0])
    
OBS_unforced_trend_pattern_correlation_30yr = []
for i in range(len(trend_annual_da['30yr'].run)):
    OBS_unforced_trend_pattern_correlation_30yr.append(stats.pearsonr(ACCESS_unforced_30yr_trend.isel(run=i).values.flatten(), Obs_ACCESS_unforced_30yr_trend['tas'].values.flatten())[0])
    
OBS_unforced_trend_pattern_correlation_60yr = []
for i in range(len(trend_annual_da['60yr'].run)):
    OBS_unforced_trend_pattern_correlation_60yr.append(stats.pearsonr(ACCESS_unforced_60yr_trend.isel(run=i).values.flatten(), Obs_ACCESS_unforced_60yr_trend['tas'].values.flatten())[0])

In [ ]:
OBS_unforced_trend_pattern_correlation_30yr

In [ ]:
# Save to a text file
with open('obsUnforced_ACCESSunforced_pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")